In [1]:
from sde_2d_v01 import Sde_2d, Heston
import numpy as np

In [2]:
'''=========
geometric asian option class
=============='''
class geometric_asian_option:
    def __init__(self,
                 otype = 1,
                 maturity = 2/12,
                 strike = 101,
                 num_time_step = 8
                ):
        self.otype = otype
        self.maturity = maturity
        self.strike = strike
        self.num_time_step = num_time_step
        
    def time_step(self):
        return np.linspace(0, maturity, num_time_step+1)
    
    #Input:
        #state_array is 1-d array with length num_time_step+1
        #all elements must be non-negative as price
    def payoff(self, price_array): 
        if len(price_array)!= self.num_time_step+1:
            print('error: price list length does not match!')
        else:
            geometric_average = np.power(np.prod(price_array), 1/(self.num_time_step+1))    
            return np.max([self.otype*(geometric_average-self.strike), 0])

In [3]:
'''============
initiate option class list
=============='''
strike_list = [97,99,101,103]
maturity = 2/12
option_list = []

for k in strike_list:
    option1 = geometric_asian_option(maturity=maturity, strike=k, num_time_step=8)
    option_list.append(option1)

In [4]:
'''================
initiate heston instance
================='''
heston1 = Heston(init_state=[100., .04], r=.05, 
                 kappa=1.2, theta=.04, xi=.3, rho=.5)

In [5]:
'''=================
- generate multiple euler paths
    - $num_path$: number of paths 
    - $num_path_time_step$: number of time steps for each path, which satifies
        - $num_path_time_step = geometric_asian_option.num_time_step* m$ for some constant $m$
- for each path, extract monitored prices, 
=================='''
num_monitored_price = option_list[0].num_time_step
m = 10 #time step refinement constant
num_path_time_step = num_monitored_price*m
num_path = 100
path_time_step = np.linspace(0, maturity, num_path_time_step+1)

path_list = [heston1.euler(path_time_step)[0] for i in range(num_path)]
indice = m*np.arange(0, num_monitored_price+1)
monitored_price_list = [path_list[i][indice] for i in range(num_path)]

#option_list[0].payoff(monitored_price_list[0])

In [6]:
'''=============
compute option price by monte carlo
==================='''

discount_factor = np.exp(-heston1.r*option_list[0].maturity)
price_list = []
for option1 in option_list:
    payoff_list = [option1.payoff(monitored_price) for monitored_price in monitored_price_list]
    price1 = discount_factor*np.mean(payoff_list)
    price_list.append(price1)
    
    

In [7]:
price_list

[3.7925736094875964,
 2.5429985313737244,
 1.6026613800895042,
 0.9366198675047346]